In [ ]:
import os
import cv2
import imutils
import dlib
from imutils import face_utils
import winsound

frequency = 2500
duration = 1000

def eyeAspectRatio(eye):
    A = cv2.norm(eye[1] - eye[5])
    B = cv2.norm(eye[2] - eye[4])
    C = cv2.norm(eye[0] - eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

def mouthAspectRatio(mouth):
    A = cv2.norm(mouth[1] - mouth[7])
    B = cv2.norm(mouth[2] - mouth[6])
    C = cv2.norm(mouth[0] - mouth[4])
    mar = (A + B) / (2.0 * C)
    return mar

earThresh = 0.3  # Distance between vertical eye coordinate threshold
marThresh = 0.3  # Threshold for mouth aspect ratio indicating yawning

# Path to the shape predictor file
shapePredictor = "C:\\Users\\dhana\\OneDrive\\Desktop\\shape_predictor_68_face_landmarks.dat"

# Load the facial landmarks predictor
predictor = dlib.shape_predictor(shapePredictor)

# Create a face detector
detector = dlib.get_frontal_face_detector()

# Indices for left and right eyes
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]
(mStart, mEnd) = face_utils.FACIAL_LANDMARKS_IDXS["mouth"]

# Path to your dataset directory
dataset_path = "C:\\Users\\dhana\\OneDrive\\Desktop\\images_folder"

# Loop through images in the dataset directory
for filename in os.listdir(dataset_path):
    # Read the image
    image_path = os.path.join(dataset_path, filename)
    frame = cv2.imread(image_path)
    
    # Check if the image is successfully loaded
    if frame is None:
        print(f"Failed to load image: {image_path}")
        continue
    
    # Perform resizing for better processing
    frame = imutils.resize(frame, width=450)
    
    # Convert the image to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Use the face detector to identify faces in the grayscale image
    rects = detector(gray, 0)
    
    for rect in rects:
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)
    
        leftEye = shape[lStart:lEnd]
        rightEye = shape[rStart:rEnd]
        leftEAR = eyeAspectRatio(leftEye)
        rightEAR = eyeAspectRatio(rightEye)
    
        ear = (leftEAR + rightEAR) / 2.0
        
        mouth = shape[mStart:mEnd]
        mar = mouthAspectRatio(mouth)
    
        leftEyeHull = cv2.convexHull(leftEye)
        rightEyeHull = cv2.convexHull(rightEye)
        cv2.drawContours(frame, [leftEyeHull], -1, (0, 0, 255), 1)
        cv2.drawContours(frame, [rightEyeHull], -1, (0, 0, 255), 1)
        
        mouthHull = cv2.convexHull(mouth)
        cv2.drawContours(frame, [mouthHull], -1, (0, 255, 0), 1)
    
        if ear < earThresh:
            cv2.putText(frame, "EYES BLINKED", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            winsound.Beep(frequency, duration)
        
        if mar > marThresh:
            cv2.putText(frame, "YAWNING DETECTED", (10, 60),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            winsound.Beep(frequency, duration)
    
    cv2.imshow("Frame", frame)
    cv2.waitKey(0)

cv2.destroyAllWindows()


In [ ]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import cv2
import dlib
from imutils import face_utils
import numpy as np
from scipy.spatial import distance as dist


frequency = 2500
duration = 1000

def eyeAspectRatio(eye):
    A = np.linalg.norm(eye[1] - eye[5])
    B = np.linalg.norm(eye[2] - eye[4])
    C = np.linalg.norm(eye[0] - eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

def mouthAspectRatio(mouth):
    A = np.linalg.norm(mouth[2] - mouth[10])
    B = np.linalg.norm(mouth[4] - mouth[8])
    C = np.linalg.norm(mouth[0] - mouth[6])
    mar = (A + B) / (2.0 * C)
    return mar

class DetectionApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Detection App")

        self.image_path = None
        self.eye_blinking_results = []
        self.yawning_results = []

        self.label = tk.Label(root, text="Detection App", font=("Helvetica", 16))
        self.label.pack(pady=10)

        self.load_button = tk.Button(root, text="Load Image", command=self.load_image)
        self.load_button.pack(pady=10)

        self.detect_eye_blinking_button = tk.Button(root, text="Detect Eye Blinking", command=self.detect_eye_blinking)
        self.detect_eye_blinking_button.pack(pady=5)

        self.detect_yawning_button = tk.Button(root, text="Detect Yawning", command=self.detect_yawning)
        self.detect_yawning_button.pack(pady=5)

        self.canvas = tk.Canvas(root, width=500, height=400)
        self.canvas.pack()

    def load_image(self):
        file_path = filedialog.askopenfilename(filetypes=[("C:\\Users\\dhana\\OneDrive\\Desktop\\images_folder", "*.jpg;*.png;*.jpeg")])
        if file_path:
            self.image_path = file_path
            self.display_image()

    def detect_eye_blinking(self):
        if self.image_path:
            frame = cv2.imread(self.image_path)
            if frame is not None:
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                rects = detector(gray, 0)
                eye_blinking_detected = False

                for rect in rects:
                    shape = predictor(gray, rect)
                    shape = face_utils.shape_to_np(shape)

                    leftEye = shape[36:42]
                    rightEye = shape[42:48]

                    leftEAR = eyeAspectRatio(leftEye)
                    rightEAR = eyeAspectRatio(rightEye)

                    ear = (leftEAR + rightEAR) / 2.0

                    if ear < 0.2:  # Example threshold for eye blinking
                        eye_blinking_detected = True
                        break

                self.eye_blinking_results.append(eye_blinking_detected)
                print("Eye Blinking Detected:", eye_blinking_detected)

    def detect_yawning(self):
        if self.image_path:
            frame = cv2.imread(self.image_path)
            if frame is not None:
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                rects = detector(gray, 0)
                yawning_detected = False

                for rect in rects:
                    shape = predictor(gray, rect)
                    shape = face_utils.shape_to_np(shape)

                    mouth = shape[48:68]
                    mar = mouthAspectRatio(mouth)

                    if mar < 0.4:  # Example threshold for yawning
                        yawning_detected = True
                        break

                self.yawning_results.append(yawning_detected)
                print("Yawning Detected:", yawning_detected)

    def display_image(self):
        if self.image_path:
            img = Image.open(self.image_path)
            img = img.resize((500, 400), Image.ANTIALIAS)
            img_tk = ImageTk.PhotoImage(img)

            self.canvas.config(width=img_tk.width(), height=img_tk.height())
            self.canvas.create_image(0, 0, anchor=tk.NW, image=img_tk)
            self.canvas.image = img_tk

if __name__ == "__main__":
    dist = __import__("scipy.spatial.distance").distance
    shape_predictor_path =  "C:\\Users\\dhana\\OneDrive\\Desktop\\shape_predictor_68_face_landmarks.dat"
    predictor = dlib.shape_predictor(shape_predictor_path)
    detector = dlib.get_frontal_face_detector()

    root = tk.Tk()
    app = DetectionApp(root)
    root.mainloop()